## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Mataura,NZ,2022-05-02 00:18:14,-46.1927,168.8643,62.47,72,13,4.18,few clouds
1,1,Cherskiy,RU,2022-05-02 00:18:15,68.7500,161.3000,28.45,85,16,11.72,few clouds
2,2,Ancud,CL,2022-05-02 00:18:15,-41.8697,-73.8203,50.32,91,41,4.14,scattered clouds
3,3,Wanning,CN,2022-05-02 00:18:16,18.8003,110.3967,69.58,94,100,16.08,overcast clouds
4,4,Jasper,US,2022-05-02 00:18:16,37.2001,-94.3502,71.11,42,40,5.75,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Jasper,US,2022-05-02 00:18:16,37.2001,-94.3502,71.11,42,40,5.75,scattered clouds
9,9,Saint-Philippe,RE,2022-05-02 00:10:19,-21.3585,55.7679,74.79,84,2,2.91,clear sky
14,14,Souillac,MU,2022-05-02 00:18:20,-20.5167,57.5167,71.98,94,20,1.14,few clouds
17,17,Kaeo,NZ,2022-05-02 00:18:21,-35.1000,173.7833,70.32,75,83,8.77,broken clouds
37,37,Mahebourg,MU,2022-05-02 00:18:29,-20.4081,57.7000,73.26,94,20,1.14,few clouds
41,41,Lompoc,US,2022-05-02 00:18:31,34.6391,-120.4579,72.84,68,0,28.99,clear sky
51,51,Ballina,AU,2022-05-02 00:19:35,-28.8667,153.5667,73.36,71,40,14.97,scattered clouds
52,52,Marawi,PH,2022-05-02 00:19:35,7.9986,124.2928,74.73,87,99,0.51,overcast clouds
55,55,Namibe,AO,2022-05-02 00:19:36,-15.1961,12.1522,73.94,68,50,8.63,scattered clouds
62,62,Banavar,IN,2022-05-02 00:19:39,13.4136,76.1683,71.55,83,84,5.28,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))

80


In [6]:
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        2
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities2_df=preferred_cities_df.dropna(axis="index", how="any")
print(len(preferred_cities2_df))

78


In [8]:
preferred_cities2_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities2_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Jasper,US,71.11,scattered clouds,37.2001,-94.3502,
9,Saint-Philippe,RE,74.79,clear sky,-21.3585,55.7679,
14,Souillac,MU,71.98,few clouds,-20.5167,57.5167,
17,Kaeo,NZ,70.32,broken clouds,-35.1000,173.7833,
37,Mahebourg,MU,73.26,few clouds,-20.4081,57.7000,
41,Lompoc,US,72.84,clear sky,34.6391,-120.4579,
51,Ballina,AU,73.36,scattered clouds,-28.8667,153.5667,
52,Marawi,PH,74.73,overcast clouds,7.9986,124.2928,
55,Namibe,AO,73.94,scattered clouds,-15.1961,12.1522,
62,Banavar,IN,71.55,broken clouds,13.4136,76.1683,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
print(len(hotel_df))
hotel_df.dtypes
hotel_df.isna().sum()

78


City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel2_df=hotel_df.dropna(axis="index", how="any")
print(len(hotel2_df))

78


In [13]:
hotel2_df


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Jasper,US,71.11,scattered clouds,37.2001,-94.3502,Guest House Motel
9,Saint-Philippe,RE,74.79,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
14,Souillac,MU,71.98,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
17,Kaeo,NZ,70.32,broken clouds,-35.1000,173.7833,Bushido Lodge
37,Mahebourg,MU,73.26,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
...,...,...,...,...,...,...,...
719,Saint-Pierre,RE,74.88,clear sky,-21.3393,55.4781,Alize Plage
723,Panjakent,TJ,70.59,overcast clouds,39.4952,67.6093,Salom Hostel
725,Rincon,US,73.83,heavy intensity rain,32.2960,-81.2354,Days Inn Rincon
738,Thai Nguyen,VN,71.38,scattered clouds,21.5928,105.8442,East Asia Hotel 2


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel2_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel2_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel2_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))